In [27]:
# Manipulación de datos
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import zipfile
import Funciones_Kaggle_ReconocimientoFacial as fc

# Carga y preprocesamiento de imágenes
import os
import cv2

# Construcción y entrenamiento del modelo
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, RMSprop

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


Vamos a comenzar descomprimiendo el archivo y luego exploraremos los datos para entender mejor el contenido y la estructura. Luego, procederemos con el preprocesamiento y el entrenamiento del modelo.

Descomprimamos el archivo primero.

In [2]:
zip_file_path = 'data.zip'
extracted_folder_path = 'data/facial_expressions'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

extracted_contents = os.listdir(extracted_folder_path)
extracted_contents

['data']

El archivo ZIP contiene una carpeta llamada data. Vamos a explorar más a fondo para ver cómo están organizadas las imágenes dentro de esta carpeta.

In [3]:
data_folder_path = os.path.join(extracted_folder_path, 'data')
data_contents = os.listdir(data_folder_path)
data_contents

['images']

Dentro de es encontramos una subcarpeta llamada images. Vamos a explorar esta subcarpeta para ver cómo están organizadas las imágenes y obtener una idea del número de imágenes disponibles.

In [4]:
images_folder_path = os.path.join(data_folder_path, 'images')
images_contents = os.listdir(images_folder_path)
num_images = len(images_contents)
images_contents[:10], num_images

(['test', 'train'], 2)

La carpeta images contiene dos subcarpetas: train y test. Exploraremos ambas carpetas, comencemos con Train.

In [5]:
train_folder_path = os.path.join(images_folder_path, 'train')
train_contents = os.listdir(train_folder_path)
num_train_images = len(train_contents)
train_contents[:10], num_train_images

(['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'], 7)

Dado que nuestro enfoque está en clasificar entre "feliz" y "triste", utilizaremos únicamente las carpetas happy y sad.

In [6]:
happy_train_folder = os.path.join(train_folder_path, 'happy')
sad_train_folder = os.path.join(train_folder_path, 'sad')

happy_train_images = os.listdir(happy_train_folder)
sad_train_images = os.listdir(sad_train_folder)

num_happy_train_images = len(happy_train_images)
num_sad_train_images = len(sad_train_images)

num_happy_train_images, num_sad_train_images

(7164, 4938)

La carpeta de entrenamiento contiene 7164 imágenes de personas felices y 4938 imágenes de personas tristes. 

Ahora exploraremos Test.

In [7]:
test_folder_path = os.path.join(images_folder_path, 'test')
test_contents = os.listdir(test_folder_path)
test_contents

['10004.jpg',
 '10019.jpg',
 '10023.jpg',
 '10029.jpg',
 '1003.jpg',
 '10031.jpg',
 '10033.jpg',
 '10043.jpg',
 '10044.jpg',
 '10048.jpg',
 '10052.jpg',
 '10053.jpg',
 '10056.jpg',
 '10065.jpg',
 '10068.jpg',
 '10073.jpg',
 '10074.jpg',
 '10079.jpg',
 '1008.jpg',
 '10095.jpg',
 '10096.jpg',
 '10097.jpg',
 '10099.jpg',
 '101.jpg',
 '10106.jpg',
 '10114.jpg',
 '10116.jpg',
 '10117.jpg',
 '10118.jpg',
 '10121.jpg',
 '10126.jpg',
 '10134.jpg',
 '10138.jpg',
 '10141.jpg',
 '10148.jpg',
 '10150.jpg',
 '10162.jpg',
 '10163.jpg',
 '10171.jpg',
 '10172.jpg',
 '10176.jpg',
 '10185.jpg',
 '10189.jpg',
 '1020.jpg',
 '10215.jpg',
 '10218.jpg',
 '1022.jpg',
 '10237.jpg',
 '1024.jpg',
 '10246.jpg',
 '10247.jpg',
 '10248.jpg',
 '10252.jpg',
 '10257.jpg',
 '10259.jpg',
 '1026.jpg',
 '10263.jpg',
 '10266.jpg',
 '10267.jpg',
 '10268.jpg',
 '1027.jpg',
 '10273.jpg',
 '10276.jpg',
 '10278.jpg',
 '10286.jpg',
 '10292.jpg',
 '10306.jpg',
 '10312.jpg',
 '10315.jpg',
 '10317.jpg',
 '1033.jpg',
 '10336.jpg',
 '

contiene una gran cantidad de imágenes, pero no están organizadas en subcarpetas de emociones. Dado que la clasificación entre "feliz" y "triste" debe realizarse, será necesario que evaluemos cómo están etiquetadas estas imágenes o si requieren una clasificación manual para el test.

Para proceder con la construcción de nuestro modelo, primero prepararemos el conjunto de entrenamiento. Realizaremos los siguientes pasos:

1. Cargar las imágenes de las carpetas happy y sad.
2. Preprocesar las imágenes (normalización, redimensionamiento si es necesario).
3. Crear las etiquetas correspondientes para cada categoría (0 para "sad" y 1 para "happy").
4. Entrenar un modelo de red neuronal convolucional (CNN) utilizando estos datos.

Vamos a comenzar con la carga y preprocesamiento de las imágenes de entrenamiento, para ello usaré el módulo de Funciones (fc).

In [8]:
happy_train_images, happy_train_labels = fc.load_images_from_folder(happy_train_folder, label=1)

In [9]:
sad_train_images, sad_train_labels = fc.load_images_from_folder(sad_train_folder, label=0)

In [45]:
images = np.array(happy_train_images + sad_train_images)
labels = np.array(happy_train_labels + sad_train_labels)

ValueError: operands could not be broadcast together with shapes (7164,48,48,1) (4938,48,48,1) 

In [10]:
X = np.concatenate((happy_train_images, sad_train_images), axis=0)
y = np.concatenate((happy_train_labels, sad_train_labels), axis=0)

In [12]:
X, y = shuffle(X, y, random_state=42)

In [13]:
X.shape, y.shape

((12102, 48, 48, 1), (12102,))

In [14]:
# Reshape the images for model compatibility
X = X.reshape(X.shape[0], 48, 48, 1)

In [15]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Convert labels to categorical
y_train_cat = to_categorical(y_train, num_classes=2)
y_val_cat = to_categorical(y_val, num_classes=2)

print(X_train.shape, y_train_cat.shape, X_val.shape, y_val_cat.shape)

(9681, 48, 48, 1) (9681, 2) (2421, 48, 48, 1) (2421, 2)


--------------------------------------------------------------------------------------------------------------------------

## Construcción y Entrenamiento del Modelo

---------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Definir el pipeline con diferentes modelos
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=50)),
    ('classifier', svm_model)  # placeholder
])

In [17]:
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'dropout_rate': [0.3, 0.4, 0.5],
    'batch_size': [32, 64],
    'epochs': [10, 20]
}


In [18]:
def create_cnn_model(optimizer='adam', dropout_rate=0.5):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate),
        
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Dropout(dropout_rate),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(dropout_rate),
        Dense(2, activation='softmax')
    ])
    
    if optimizer == 'adam':
        opt = Adam()
    elif optimizer == 'rmsprop':
        opt = RMSprop()
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [20]:
def cnn_grid_search(X_train, y_train, param_grid, cv=3):
    best_score = 0
    best_params = {}
    
    for optimizer in param_grid['optimizer']:
        for dropout_rate in param_grid['dropout_rate']:
            for batch_size in param_grid['batch_size']:
                for epochs in param_grid['epochs']:
                    print(f"Training with optimizer={optimizer}, dropout_rate={dropout_rate}, batch_size={batch_size}, epochs={epochs}")
                    
                    model = create_cnn_model(optimizer=optimizer, dropout_rate=dropout_rate)
                    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2, verbose=1)
                    
                    val_accuracy = max(history.history['val_accuracy'])
                    if val_accuracy > best_score:
                        best_score = val_accuracy
                        best_params = {
                            'optimizer': optimizer,
                            'dropout_rate': dropout_rate,
                            'batch_size': batch_size,
                            'epochs': epochs
                        }
    
    return best_score, best_params

In [21]:
best_score, best_params = cnn_grid_search(X_train, y_train_cat, param_grid)
print(f"Best Score: {best_score}")
print(f"Best Params: {best_params}")

Training with optimizer=adam, dropout_rate=0.3, batch_size=32, epochs=10


C:\Users\buque\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 12s 38ms/step - accuracy: 0.5802 - loss: 0.6811 - val_accuracy: 0.6242 - val_loss: 0.6604
Epoch 2/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.6311 - loss: 0.6391 - val_accuracy: 0.6892 - val_loss: 0.5685
Epoch 3/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.7139 - loss: 0.5547 - val_accuracy: 0.7656 - val_loss: 0.4862
Epoch 4/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.7433 - loss: 0.5031 - val_accuracy: 0.7723 - val_loss: 0.4694
Epoch 5/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - accuracy: 0.7752 - loss: 0.4621 - val_accuracy: 0.8090 - val_loss: 0.4130
Epoch 6/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.7913 - loss: 0.4381 - val_accuracy: 0.8121 - val_loss: 0.3868
Epoch 7/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 10s 43ms/step - accuracy: 0.8072 - loss: 0.4124 - val_accuracy: 0.8250 - val_loss: 0.3828
Epoch 8/10
242/242 ━━━━━━━━━━━━━━━━━━━━ 12s 48ms/step - accuracy: 0.8159 - loss: 0.3931 - 

In [22]:
# Train the final model with the best hyperparameters
best_model = create_cnn_model(optimizer=best_params['optimizer'], dropout_rate=best_params['dropout_rate'])
best_model.fit(X_train, y_train_cat, batch_size=best_params['batch_size'], epochs=best_params['epochs'], validation_data=(X_val, y_val_cat), verbose=1)

Epoch 1/20


C:\Users\buque\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


303/303 ━━━━━━━━━━━━━━━━━━━━ 16s 47ms/step - accuracy: 0.5860 - loss: 0.6887 - val_accuracy: 0.6993 - val_loss: 0.5821
Epoch 2/20
303/303 ━━━━━━━━━━━━━━━━━━━━ 14s 47ms/step - accuracy: 0.6965 - loss: 0.5754 - val_accuracy: 0.7798 - val_loss: 0.4589
Epoch 3/20
303/303 ━━━━━━━━━━━━━━━━━━━━ 14s 46ms/step - accuracy: 0.7591 - loss: 0.4782 - val_accuracy: 0.7947 - val_loss: 0.4390
Epoch 4/20
303/303 ━━━━━━━━━━━━━━━━━━━━ 15s 48ms/step - accuracy: 0.7814 - loss: 0.4361 - val_accuracy: 0.8125 - val_loss: 0.3997
Epoch 5/20
303/303 ━━━━━━━━━━━━━━━━━━━━ 14s 46ms/step - accuracy: 0.8167 - loss: 0.3911 - val_accuracy: 0.8344 - val_loss: 0.3553
Epoch 6/20
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.8229 - loss: 0.3773 - val_accuracy: 0.8278 - val_loss: 0.3676
Epoch 7/20
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 44ms/step - accuracy: 0.8375 - loss: 0.3593 - val_accuracy: 0.8340 - val_loss: 0.3505
Epoch 8/20
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 43ms/step - accuracy: 0.8408 - loss: 0.3473 - val_accurac

In [31]:
X_test, test_filenames = fc.load_images_from_folder(test_folder_path, label=1)

In [32]:
predictions = best_model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

221/221 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step


In [42]:
submission = pd.DataFrame({
    'id_img': list(range(len(test_filenames))),
    'label': ['happy' if label == 1 else 'sad' for label in predicted_labels]
})

submission.to_csv('submission.csv', index=False)
submission

,id_img,label
0,0,sad
1,1,happy
2,2,sad
3,3,sad
4,4,sad
...,...,...
7061,7061,happy
7062,7062,happy
7063,7063,happy
7064,7064,happy


In [43]:
submission = submission.drop_duplicates(subset=['id_img'])

In [44]:
submission

,id_img,label
0,0,sad
1,1,happy
2,2,sad
3,3,sad
4,4,sad
...,...,...
7061,7061,happy
7062,7062,happy
7063,7063,happy
7064,7064,happy
